In [2]:
import pandas as pd
import os

In [3]:
os.getcwd()

'/home/nina/PycharmProjects/INSIDe/inside_demonstrator/analysis'

In [150]:
experiment_name = "final_tests/3_MittelstarkerRegen/b_constant_hydraulics_same"

"""
final_tests/1_KeinRegen/0_recalculate_hydraulics
final_tests/2_Nieselregen/a_constant_hydraulics_avg
final_tests/3_MittelstarkerRegen/b_constant_hydraulics_same
"""

'\nfinal_tests/1_KeinRegen/0_recalculate_hydraulics\nfinal_tests/2_Nieselregen/a_constant_hydraulics_avg\nfinal_tests/3_MittelstarkerRegen/b_constant_hydraulics_same\n'

In [151]:
data_directory = f"../tandler_output/preprocessed/{experiment_name}"
try:
    os.makedirs(data_directory)
except:
    print("Folder structure already exists")

Folder structure already exists


### INSIDe substance results

In [153]:
file_directory = f"../tandler_output/original/{experiment_name}/"
sampling_files = [el for el in os.listdir(file_directory) if "Sampling_COV_INSIDe" in el]
sampling_curves_files = [el for el in os.listdir(file_directory) if (("INSIDe_substances_results" in el) and ("Sampling" not in el))]
sampling_curves_files

['INSIDe_substances_results_2024_01_07__19_42_17_manhole_MW043.txt',
 'INSIDe_substances_results_2024_01_07__19_42_17_manhole_RW141.txt',
 'INSIDe_substances_results_2024_01_07__19_42_17_manhole_MW022.txt',
 'INSIDe_substances_results_2024_01_07__19_42_17_manhole_RW157.txt',
 'INSIDe_substances_results_2024_01_07__19_42_17_manhole_MW059.txt',
 'INSIDe_substances_results_2024_01_07__19_42_17_manhole_MW017.txt',
 'INSIDe_substances_results_2024_01_07__19_42_17_manhole_RW126.txt',
 'INSIDe_substances_results_2024_01_07__19_42_17_manhole_RW155.txt',
 'INSIDe_substances_results_2024_01_07__19_42_17_manhole_RW143.txt',
 'INSIDe_substances_results_2024_01_07__19_42_17_manhole_MW064.txt',
 'INSIDe_substances_results_2024_01_07__19_42_17_manhole_MW052.txt',
 'INSIDe_substances_results_2024_01_07__19_42_17_manhole_MW048.txt',
 'INSIDe_substances_results_2024_01_07__19_42_17_manhole_RW211.txt',
 'INSIDe_substances_results_2024_01_07__19_42_17_manhole_RW156.txt',
 'INSIDe_substances_results_2024_0

In [154]:
def tidy_concentration_curve(path, CSB_and_temp=False):
    # load the complete dataset
    df = pd.read_csv(path, sep="\t")
    df.rename({"time[min]": "time"}, axis=1, inplace=True)

    sampling_location = path.split("_manhole_")[1].replace(".txt", "")

    # identify positions of new subdatasets
    subdataframes = df.loc[df["time"]=="time[min]"]

    if CSB_and_temp:
        # CSB (Chemischer Sauerstoffbedarf) concentration
        df_1 = df.iloc[:subdataframes.index[0]].copy()
        df_1["location"] = sampling_location
         # T(Temperature) concentration
        df_3 = df.iloc[subdataframes.index[1]:subdataframes.index[2]].copy()
        df_3.columns = df_3.iloc[0]
        df_3 = df_3.iloc[1:]
        df_3.reset_index(drop=True, inplace=True)
        df_3.rename({"time[min]": "time"}, axis=1, inplace=True)
        df_3["location"] = sampling_location

        df_4 = df.iloc[subdataframes.index[2]:].copy()
        df_4.columns = df_4.iloc[0]
        df_4 = df_4.iloc[1:]
        df_4.reset_index(drop=True, inplace=True)
        df_4.rename({"time[min]": "time"}, axis=1, inplace=True)
        df_4["location"] = sampling_location
        df_2 = df.iloc[subdataframes.index[0]:subdataframes.index[1]].copy()
         # COV(COV19) concentration 
        df_2.columns = df_2.iloc[0]
        df_2 = df_2.iloc[1:]
        df_2.reset_index(drop=True, inplace=True)
    else:
        df_2 = df.copy()

   
    df_2.rename({"time[min]": "time"}, axis=1, inplace=True)
    df_2["location"] = sampling_location

   
    if CSB_and_temp:
        # combine the dataframes
        df_1[df_2.columns[1]] = df_2.iloc[:,1]
        df_1[df_3.columns[1]] = df_3.iloc[:,1]
        df_1[df_4.columns[1]] = df_4.iloc[:,1]
        df_1.columns = ['time', 'CSB',
       'location', 'AFS63',
       'COV19',
       'Temperature']
        return df_1
    else:
        df_2.columns = ['time', 'COV19',
       'location']
        return df_2


In [155]:
df = pd.DataFrame()
for i in range(len(sampling_curves_files)):
    path = f"{file_directory}{sampling_curves_files[i]}"
    df_sub = tidy_concentration_curve(path)
    df = pd.concat((df, df_sub))
df.reset_index(inplace=True, drop=True)

In [156]:
df

,time,COV19,location
0,-10,0.0,MW043
1,-5,0.0,MW043
2,0,0.0,MW043
3,5,0.0,MW043
4,10,0.0,MW043
...,...,...,...
72625,20140,0.1,MW046
72626,20145,0.2,MW046
72627,20150,0.2,MW046
72628,20155,0.1,MW046


In [157]:
df["location"].unique()

array(['MW043', 'RW141', 'MW022', 'RW157', 'MW059', 'MW017', 'RW126',
       'RW155', 'RW143', 'MW064', 'MW052', 'MW048', 'RW211', 'RW156',
       'MW061', 'MW054', 'MW023', 'MW046'], dtype=object)

In [158]:
df.to_csv(data_directory+"/concentration_curves.csv", index=False)

### Flow rates

In [159]:
experiment_name

'final_tests/3_MittelstarkerRegen/b_constant_hydraulics_same'

In [160]:
file_directory = f"../tandler_output/original/{experiment_name}/"
flow_files = [el for el in os.listdir(file_directory) if "flow_rate" in el]
flow_files = [el for el in flow_files if ("discharge" not in el)]
flow_files[1:3]

['INSIDe_hydraulic_results_2024_01_07__18_50_18_pipe_RW219_flow_rate.txt',
 'INSIDe_hydraulic_results_2024_01_07__19_39_29_pipe_RW214_flow_rate.txt']

In [161]:
def tidy_flow_rate_curve(path):
    # load the complete dataset
    df = pd.read_csv(path, sep="\t")
    df.rename({"t [min] ": "time", " flow rate [l/s]": "flow_rate"}, axis=1, inplace=True)
    df["location"] = path.split("pipe_")[1].split("_flow_rate")[0]
    return df

df = pd.DataFrame()
for i in range(len(flow_files)):
    path = f"{file_directory}{flow_files[i]}"
    df_sub = tidy_flow_rate_curve(path)
    df = pd.concat((df, df_sub))
df.reset_index(inplace=True, drop=True)

In [162]:
df["location"].unique()

array(['RW144', 'RW219', 'RW214', 'RW139', 'MW025', '123', 'MW056',
       'MW015', 'RW130', 'MW048', 'MW014', 'RW220', 'RW129', 'RW127',
       'RW215', 'MW057', 'MW050', 'MW069', 'MW051', 'MW076', '', 'RW140',
       'MW053', 'RW221', 'MW033', 'RW135', 'RW155', 'MW020', 'MW058',
       'MW059', 'MW077', 'RW134', 'MW018', 'RW132', 'RW216', 'MW054',
       'MW021', 'MW055', 'MW070', 'RW146', 'RW223', 'RW222', 'MW029',
       'MW023', 'MW074', 'MW047', 'RW143', 'RW157', 'MW043', 'MW073',
       'RW156', 'MW017', 'MW000', 'RW133', 'RW131', 'MW049', 'RW158',
       'RW138', 'MW061', 'RW160', 'MW022', 'RW145', 'MW024', 'RW152',
       'RW161', 'RW154', 'RW137', 'RW153', 'RW224', 'MW046', 'RW213',
       'RW218', 'MW045', 'MW060', 'RW212', '456', 'RW159', 'RW141',
       'MW075', 'RW142', 'RW147', 'MW071', 'MW016', 'MW052', 'MW038',
       'RW128', 'RW136', 'RW217', 'MW072', 'MW063'], dtype=object)

In [163]:
df.to_csv(data_directory+"/flow_rates.csv", index=False)